In [1]:
# Step 1: Import necessary libraries
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
import time

In [2]:
# Step 2: Data Preparation
# Define transformations for the training set
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),  # Random crop with padding
    transforms.RandomHorizontalFlip(),     # Random horizontal flip
    transforms.ToTensor(),                 # Convert to tensor
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2470, 0.2435, 0.2616))  # Normalize
])

# Download and load the CIFAR-10 dataset
batch_size = 128
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=2)

100%|██████████| 170M/170M [00:19<00:00, 8.93MB/s] 


Extracting ./data\cifar-10-python.tar.gz to ./data


In [4]:
# Step 3: Define the CNN architecture with customizable activation function
class CIFAR10CNN(nn.Module):
    def __init__(self, activation_fn):
        super(CIFAR10CNN, self).__init__()
        
        # Layer 1: Convolutional layer
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(32)
        self.activation = activation_fn
        self.pool1 = nn.MaxPool2d(kernel_size=2)
        
        # Layer 2: Convolutional layer
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(64)
        self.pool2 = nn.MaxPool2d(kernel_size=2)
        
        # Layer 3: Convolutional layer
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.bn3 = nn.BatchNorm2d(128)
        
        # Layer 4: Convolutional layer
        self.conv4 = nn.Conv2d(128, 128, kernel_size=3, padding=1)
        self.bn4 = nn.BatchNorm2d(128)
        self.pool4 = nn.MaxPool2d(kernel_size=2)
        
        # Fully connected layers
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(128 * 4 * 4, 512)
        self.dropout = nn.Dropout(0.5)
        self.fc2 = nn.Linear(512, 10)  # 10 classes for CIFAR-10
    
    def forward(self, x):
        # Layer 1
        x = self.pool1(self.activation(self.bn1(self.conv1(x))))
        # Layer 2
        x = self.pool2(self.activation(self.bn2(self.conv2(x))))
        # Layer 3
        x = self.activation(self.bn3(self.conv3(x)))
        # Layer 4
        x = self.pool4(self.activation(self.bn4(self.conv4(x))))
        # Fully connected layers
        x = self.flatten(x)
        x = self.activation(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x

In [5]:
# Step 4: Initialize device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cpu
